# Team 8 - CAP outcome

# Libraries




In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# Access to Big Query

In [0]:
auth.authenticate_user()

In [0]:
project_id='datathon-tarragona-2018'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [0]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

# Access to Google Drive

In [0]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

#Database Overview

In [0]:
# Total_unique_patients
run_query(''' SELECT COUNT(ROW_ID) AS Total_unique_patients FROM `physionet-data.mimiciii_clinical.patients`  ''')

In [0]:
# Total admissions
run_query(''' SELECT COUNT(HADM_ID) AS Total_admissions FROM `physionet-data.mimiciii_clinical.admissions`  ''')

In [0]:
df = run_query('''

  SELECT  SUBJECT_ID, ICD9_CODE, COUNT(*)  
    FROM `physionet-data.mimiciii_clinical.diagnoses_icd` 
    WHERE (SEQ_NUM BETWEEN 1 AND 2) AND (REGEXP_CONTAINS(ICD9_CODE, "^48[0-3]")) 
    GROUP BY SUBJECT_ID, ICD9_CODE
  
''')

In [0]:
# Total_unique_patients
run_query(''' SELECT COUNT(ROW_ID) AS Total_unique_patients FROM `physionet-data.mimiciii_clinical.patients`  ''')

In [0]:
#### Patients with pneumonia and age >17 Primary 480.0 - 483.99

pneu1 = run_query('''

WITH edad AS (

SELECT adm.HADM_ID , pat.GENDER, DATE_DIFF(DATE(adm.ADMITTIME), DATE(pat.dob), YEAR) AS age, pat.SUBJECT_ID
FROM `physionet-data.mimiciii_clinical.patients` AS pat INNER JOIN `physionet-data.mimiciii_clinical.admissions` AS adm 
ON pat.SUBJECT_ID = adm.SUBJECT_ID 
),
pneumonia AS (
SELECT *
FROM edad 
GROUP BY HADM_ID, GENDER, age, SUBJECT_ID
HAVING ( age > 17 )
)
SELECT diag.HADM_ID, diag.SUBJECT_ID, pneumonia.age, pneumonia.gender, diag.ICD9_CODE, diag.SEQ_NUM
  FROM `physionet-data.mimiciii_clinical.diagnoses_icd` AS diag 
  INNER JOIN pneumonia  
  ON diag.HADM_ID = pneumonia.HADM_ID
  WHERE (SEQ_NUM = 1) AND (REGEXP_CONTAINS(ICD9_CODE, "^48[0123567]")) 
  GROUP BY diag.HADM_ID, diag.SUBJECT_ID, pneumonia.age, pneumonia.gender, diag.ICD9_CODE, diag.SEQ_NUM  

''')

In [0]:
pneu1.shape
# Total 1135 patients

In [0]:
#### Patients with resp fail or sepsis with age >17 Primary 51881 - 038  Secondary 480.0 - 483.99

resp_fail_sepsis = run_query('''
WITH
  pneunm_2 AS(
  SELECT
    diag.HADM_ID,
    diag.SUBJECT_ID,
    diag.ICD9_CODE,
    diag.SEQ_NUM
  FROM
    `physionet-data.mimiciii_clinical.diagnoses_icd` AS diag
  WHERE
    SEQ_NUM = 2
    AND REGEXP_CONTAINS(ICD9_CODE, "^48[0123567]") ),
  respfail_sepsis AS (
  SELECT
    diag.HADM_ID,
    diag.SUBJECT_ID,
    diag.ICD9_CODE,
    diag.SEQ_NUM
  FROM
    `physionet-data.mimiciii_clinical.diagnoses_icd` AS diag
  WHERE
    SEQ_NUM = 1
    AND (REGEXP_CONTAINS(ICD9_CODE, "^038")
      OR REGEXP_CONTAINS(ICD9_CODE, "^51881") ) )
SELECT
  pneunm_2.SUBJECT_ID,
    pneunm_2.HADM_ID,
  pneunm_2.ICD9_CODE AS ICD_pneunm_2,
  pneunm_2.SEQ_NUM AS SEQ_pneunm_2,
      respfail_sepsis.ICD9_CODE AS ICD_respfail_sepsis,
  respfail_sepsis.SEQ_NUM AS SEQ_respfail_sepsis
FROM
  pneunm_2 INNER JOIN respfail_sepsis
  ON respfail_sepsis.HADM_ID = pneunm_2.HADM_ID
  ORDER BY HADM_ID 
  
  ''')

In [0]:
resp_fail_sepsis.shape
# Total 614 patients with Respiratory Fail or Sepsis with age >17 Primary 51881 - 038  Secondary 480.0 - 483.99

In [0]:
pneu1['HADM_ID'].head()

In [0]:
resp_fail_sepsis['HADM_ID'].head()

In [0]:
df2 = pd.concat([pneu1['HADM_ID'] , resp_fail_sepsis['HADM_ID']],  ignore_index=True)

In [0]:
df2.shape

In [0]:
df2.head()

In [0]:
# transfor to list
a= df2.iloc[1:].tolist()    

## Access to the INPUTEVENTS table which includes medication to identify Antibiotics

In [0]:
for patient_HADM in a:
  
  query = " SELECT HADM_ID , SUBJECT_ID , ITEMID, STARTTIME, AMOUNT, AMOUNTUOM, PATIENTWEIGHT, ORDERCATEGORYNAME  FROM `physionet-data.mimiciii_clinical.inputevents_mv` WHERE HADM_ID = "
  query += str(patient_HADM)
    
  dff = run_query( query )
  try:
    df7 = pd.concat([df7,dff])
  except:
    df7 = dff
  
 

In [0]:
df7.to_csv("inputevents_mv.csv", sep=',', encoding = 'iso-8859-1')
!mv inputevents_mv.csv GDrive/My\ Drive/DatathonTGN

## Access to the CHARTEVENTS table to identify gender, 

In [0]:
for patient_HADM in a:
  
  query = " SELECT SUBJECT_ID , HADM_ID , ITEMID , CHARTTIME , VALUENUM , VALUEUOM  FROM `physionet-data.mimiciii_clinical.chartevents` WHERE HADM_ID = "
  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df8 = pd.concat([df8,dff])
  except:
    df8 = dff
  

In [0]:
df8.to_csv("chartevents.csv", sep=',', encoding = 'iso-8859-1')
!mv chartevents.csv GDrive/My\ Drive/DatathonTGN

 ## Access to PROCEDUREEVENTS_MV table to filter medical procedures

In [0]:
for patient_HADM in a:
  
  query = " SELECT SUBJECT_ID , HADM_ID , STARTTIME , VALUE , VALUEUOM  FROM `physionet-data.mimiciii_clinical.procedureevents_mv`  WHERE HADM_ID = "
  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df9 = pd.concat([df9,dff])
  except:
    df9 = dff

In [0]:
df9.to_csv("procedureevents.csv", sep=',', encoding = 'iso-8859-1')
!mv procedureevents.csv GDrive/My\ Drive/DatathonTGN

## Access to the DATETIMEEVENTS table to access data ICU admission date, Date of Birth

In [0]:
for patient_HADM in a:
  
  query = " SELECT SUBJECT_ID, HADM_ID, ITEMID, CHARTTIME, VALUE, VALUEUOM FROM `physionet-data.mimiciii_clinical.datetimeevents`  WHERE HADM_ID = "
  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df10 = pd.concat([df10,dff])
  except:
    df10 = dff

In [0]:
df10.to_csv("datetimeevents.csv", sep=',', encoding = 'iso-8859-1')
!mv datetimeevents.csv GDrive/My\ Drive/DatathonTGN

## Access to the ADMISSIONS table to identify if patient died within 28 days


In [0]:
for patient_HADM in a:
  
  query = " SELECT SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME FROM `physionet-data.mimiciii_clinical.admissions` WHERE HADM_ID = "
  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df11 = pd.concat([df11,dff])
  except:
    df11 = dff

In [0]:
df11.to_csv("admissions.csv", sep=',', encoding = 'iso-8859-1')
!mv admissions.csv GDrive/My\ Drive/DatathonTGN

# Access to SOFA table to know the severity of illness

In [0]:
for patient_HADM in a:

  query = " SELECT subject_id, hadm_id, icustay_id, SOFA FROM `physionet-data.mimiciii_derived.sofa` WHERE HADM_ID = "

  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df13 = pd.concat([df13,dff])
  except:
    df13 = dff

In [0]:
df13.to_csv("sofa.csv", sep=',', encoding = 'iso-8859-1')
!mv sofa.csv GDrive/My\ Drive/DatathonTGN

# Access to ICUSTAYS table to know the LOS (lenght of stay)

In [0]:
for patient_HADM in a:

  query = " SELECT SUBJECT_ID, HADM_ID, INTIME, OUTTIME, LOS FROM `physionet-data.mimiciii_clinical.icustays` WHERE HADM_ID = "
  query += str(patient_HADM)
  
  dff = run_query( query )
  try:
    df14 = pd.concat([df14,dff])
  except:
    df14 = dff

In [0]:
df14.to_csv("icustays.csv", sep=',', encoding = 'iso-8859-1')
!mv icustays.csv GDrive/My\ Drive/DatathonTGN

# Others

In [0]:
for patient_HADM in a:
  
  query = " SELECT SUBJECT_ID , HADM_ID , ITEMID , CHARTTIME , VALUENUM , VALUEUOM  FROM `physionet-data.mimiciii_clinical.chartevents` WHERE HADM_ID = "
  query += str(patient_HADM)
  query += "AND ITEMID = 226228"
  
  dff = run_query( query )
  try:
    df15 = pd.concat([df15,dff])
  except:
    df15 = dff

In [0]:
df15.to_csv("gender.csv", sep=',', encoding = 'iso-8859-1')
!mv gender.csv GDrive/My\ Drive/DatathonTGN